In [1]:
genres = sc.textFile("../data/ML_spark/MovieLens/u.genre")

In [2]:
for line in genres.take(10):
    print line

unknown|0
Action|1
Adventure|2
Animation|3
Children's|4


In [3]:
genre_map = genres.filter(lambda x: len(x) > 0).map(lambda line : line.split('|')).map(lambda x:(x[1],x[0])).collectAsMap()

In [4]:
genre_map

{u'0': u'unknown',
 u'1': u'Action',
 u'10': u'Film-Noir',
 u'11': u'Horror',
 u'12': u'Musical',
 u'13': u'Mystery',
 u'14': u'Romance',
 u'15': u'Sci-Fi',
 u'16': u'Thriller',
 u'17': u'War',
 u'18': u'Western',
 u'2': u'Adventure',
 u'3': u'Animation',
 u'4': u"Children's",
 u'5': u'Comedy',
 u'6': u'Crime',
 u'7': u'Documentary',
 u'8': u'Drama',
 u'9': u'Fantasy'}

In [5]:
print genre_map

{u'11': u'Horror', u'10': u'Film-Noir', u'13': u'Mystery', u'12': u'Musical', u'15': u'Sci-Fi', u'14': u'Romance', u'17': u'War', u'16': u'Thriller', u'18': u'Western', u'1': u'Action', u'0': u'unknown', u'3': u'Animation', u'2': u'Adventure', u'5': u'Comedy', u'4': u"Children's", u'7': u'Documentary', u'6': u'Crime', u'9': u'Fantasy', u'8': u'Drama'}


In [6]:
movies = sc.textFile('../data/ML_spark/MovieLens/u.item')
print movies.first()

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0


In [7]:
def func1(array):
    genres = array[5:]
    genres_assigned = zip(genres, range(len(genres)))
    index_1=[]
    for x,y in genres_assigned:
        if x=='1':
            index_1.append(y)
    index_1_val = [genre_map[str(i)] for i in index_1]
    index_1_val_str = ','.join(index_1_val)
    return (int(array[0]),array[1]+','+index_1_val_str)
#     return array[0]+','+array[1]+','+
titles_and_genres = movies.map(lambda x: x.split('|')).map(lambda x:func1(x))
titles_and_genres.first()

(1, u"Toy Story (1995),Animation,Children's,Comedy")

In [8]:
titles_and_genres

PythonRDD[8] at RDD at PythonRDD.scala:43

#train the recommendation model

In [9]:
from pyspark.mllib.recommendation import ALS

In [10]:
from pyspark.mllib.recommendation import Rating

In [11]:
raw_data = sc.textFile("../data/ML_spark/MovieLens/u.data")

In [12]:
raw_ratings = raw_data.map(lambda x:x.split('\t')[:3])
print raw_ratings.first()

[u'196', u'242', u'3']


In [13]:
ratings = raw_ratings.map(lambda x: Rating(x[0], x[1], x[2]))

In [14]:
ratings.cache()

PythonRDD[12] at RDD at PythonRDD.scala:43

In [27]:
als_model = ALS.train(ratings,50,5,0.1)

In [28]:
from pyspark.mllib.linalg import Vectors

In [29]:
movie_factors = als_model.productFeatures().map(lambda (id,factor): (id,Vectors.dense(factor)))
print movie_factors.first()
movie_vectors = movie_factors.map(lambda (id,vec):vec)
user_factors = als_model.userFeatures().map(lambda (id,factor):(id,Vectors.dense(factor)))
user_vectors = user_factors.map(lambda (id, vec):vec)
print user_vectors.first()

(1, DenseVector([0.1189, 0.154, -0.1281, 0.0743, 0.3372, -0.0218, -0.1564, -0.0752, -0.3558, -0.129, -0.2035, 0.425, 0.2254, 0.0389, -0.16, 0.1132, -0.0508, -0.2512, 0.3065, -0.3016, 0.2264, -0.1025, 0.594, 0.4342, 0.0976, -0.2594, 0.4988, -0.1878, -0.543, -0.2482, -0.2286, -0.2257, -0.3169, 0.5306, -0.2114, 0.1968, 0.1103, -0.1596, 0.446, 0.13, -0.2431, -0.1562, -0.2451, 0.2605, -0.5239, -0.1533, -0.078, -0.18, 0.0902, -0.2976]))
[0.287010610104,-0.306130200624,-0.0110167916864,-0.100282646716,0.402284443378,0.133642598987,-0.17621190846,0.188554614782,-0.327551275492,-0.263691723347,-0.457682311535,0.524626433849,0.15720166266,-0.0829833373427,-0.295744478703,0.105343133211,0.277225226164,-0.273413777351,0.335160762072,-0.185756832361,0.445180237293,-0.600775659084,0.723579525948,-0.00662225764245,0.0986614897847,-0.320296704769,0.743609786034,-0.180224940181,-0.503776729107,-0.422970384359,-0.56777215004,-0.231761977077,0.00380780920386,1.10723686218,-0.27037063241,-0.0452572144568,

# Normalization

In [30]:
from pyspark.mllib.linalg.distributed import RowMatrix

In [31]:
moive_matrix = RowMatrix(movie_vectors)
user_matrix = RowMatrix(user_vectors)

In [32]:
from pyspark.mllib.stat import MultivariateStatisticalSummary
desc_moive_matrix = MultivariateStatisticalSummary(moive_matrix.rows)
desc_user_matrix = MultivariateStatisticalSummary(user_matrix.rows)
print desc_moive_matrix.mean()
print desc_user_matrix.mean()
print desc_moive_matrix.variance()
print desc_user_matrix.variance()

[ 0.18750814  0.01169254 -0.02734478 -0.170362    0.26449361  0.09328681
 -0.09368825 -0.05455146 -0.31470811 -0.11623462 -0.17735984  0.29497074
  0.01358745 -0.03821302 -0.12034573  0.04143458  0.00713291 -0.19282069
  0.25698791 -0.11527816  0.27024335 -0.23577561  0.4383614   0.07292199
  0.03154241 -0.03207364  0.34252166 -0.11187901 -0.29416202 -0.25259927
 -0.14792767 -0.03822178 -0.1166447   0.37780763 -0.11059189  0.02250264
  0.19376867 -0.18940171  0.31564529 -0.03376821 -0.15598154  0.04339252
 -0.20381016  0.04950823 -0.31326934 -0.06651079  0.09376307 -0.15284407
  0.1306952  -0.19199014]
[ 0.28232441 -0.02025034 -0.0503165  -0.20999107  0.43982959  0.135303
 -0.10612427 -0.13162832 -0.44522603 -0.17735804 -0.26152583  0.46319602
  0.04430005 -0.05514997 -0.20189153  0.05529095 -0.00535763 -0.25131599
  0.46477523 -0.19353156  0.4080227  -0.33486004  0.69500958  0.08647023
  0.06751767 -0.08725915  0.60393113 -0.1492248  -0.50321613 -0.41688217
 -0.2505031  -0.13129496 -0

#Training a clustering model

In [33]:
from pyspark.mllib.clustering import KMeans

In [34]:
num_clusters = 5
num_iterations = 20
num_runs =3

In [35]:
movie_cluster_model = KMeans.train(movie_vectors,num_clusters, num_iterations, num_runs)

In [36]:
movie_cluster_model_coverged = KMeans.train(movie_vectors,num_clusters,100)

In [37]:
user_cluster_model = KMeans.train(user_vectors,num_clusters,num_iterations, num_runs)

In [38]:
movie1 = movie_vectors.first()
movie1_cluster = movie_cluster_model.predict(movie1)
print movie1_cluster

4


In [39]:
predictions = movie_cluster_model.predict(movie_vectors)
print ",".join([str(i) for i in predictions.take(10)])

4,0,0,3,0,4,3,4,4,3


In [40]:
titles_factors = titles_and_genres.join(movie_factors)

In [41]:
# titles_and_genres.first()
movie_factors.first()
# titles_and_genres.first()
titles_factors.first()

(2,
 (u'GoldenEye (1995),Action,Adventure,Thriller',
  DenseVector([-0.0777, 0.0404, -0.1395, -0.0983, 0.2061, 0.2069, -0.2453, -0.0599, -0.2359, -0.2195, -0.2442, 0.3522, -0.1026, -0.2338, -0.1505, 0.1071, -0.1449, -0.2069, 0.2843, -0.1755, 0.3383, -0.0528, 0.4846, 0.4116, -0.1236, -0.1501, 0.2868, -0.2654, -0.4177, -0.269, -0.1112, -0.0626, -0.2138, 0.3742, -0.2308, -0.0122, 0.2077, -0.2632, 0.4234, -0.0047, 0.0641, 0.2155, -0.214, 0.2074, -0.4964, -0.0304, -0.1361, -0.132, 0.0544, -0.198])))

In [42]:
def func2(x):
    id,(name_genres,vec) = x
    pred = movie_cluster_model.predict(vec)
    cluster_center = movie_cluster_model.clusterCenters[pred]
    cluster_center_vec = Vectors.dense(cluster_center)
    
#     return str(id)+','+str(pred)+','+name_genres+','+str(dist)
    dist = vec.squared_distance(cluster_center_vec)
    return str(id)+','+ name_genres+','+str(pred)+','+str(dist)

movies_assigned = titles_factors.map(lambda x:func2(x))  

In [43]:
movies_assigned.first()

u'2,GoldenEye (1995),Action,Adventure,Thriller,0,0.533530635748'

In [44]:
movies_assigned.take(20)

[u'2,GoldenEye (1995),Action,Adventure,Thriller,0,0.533530635748',
 u'4,Get Shorty (1995),Action,Comedy,Drama,3,0.860696969886',
 u'6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Drama,4,2.52523804068',
 u"8,Babe (1995),Children's,Comedy,Drama,4,0.993965215446",
 u'10,Richard III (1995),Drama,War,3,1.38891969026',
 u'12,Usual Suspects, The (1995),Crime,Thriller,4,0.892611987906',
 u'14,Postino, Il (1994),Drama,Romance,4,1.06500921868',
 u'16,French Twist (Gazon maudit) (1995),Comedy,Romance,3,1.22869754205',
 u'18,White Balloon, The (1995),Drama,3,1.31161874418',
 u'20,Angels and Insects (1995),Drama,Romance,4,2.08422762873',
 u'22,Braveheart (1995),Action,Drama,War,0,1.04398419241',
 u'24,Rumble in the Bronx (1995),Action,Adventure,Crime,3,1.0724510624',
 u'26,Brothers McMullen, The (1995),Comedy,4,1.29409091658',
 u'28,Apollo 13 (1995),Action,Drama,Thriller,0,0.513403191581',
 u'30,Belle de jour (1967),Drama,4,0.772418653435',
 u'32,Crumb (1994),Documentary,3,0.964802235433',

#coputing performance metrics

In [45]:
movie_cost = movie_cluster_model.computeCost(movie_vectors)
user_cost = user_cluster_model.computeCost(user_vectors)
print "WCSS for movies: %f"%movie_cost
print "WCSS for users: %f"%user_cost

WCSS for movies: 2172.650469
WCSS for users: 1458.771774


In [51]:
train_test_split_movies = movie_vectors.randomSplit([0.6,0.4],123)
train_movies = train_test_split_movies[0]
test_movies = train_test_split_movies[1]
for k in [2,3,4,5,10,20]:
    k_model = KMeans.train(train_movies,num_iterations,k,num_runs)
    cost = k_model.computeCost(test_movies)
    print 'WCSS for k=%d : %f'%(k,cost)
# costs_moives = [2,3,4,5,10,20].map(lambda k:(k, KMeans.train(train_movies,num_iterations,k,num_runs)).compute_cost(test_movies))

WCSS for k=2 : 790.686228
WCSS for k=3 : 785.881720
WCSS for k=4 : 784.198163
WCSS for k=5 : 788.684923
WCSS for k=10 : 771.914133
WCSS for k=20 : 778.678835


In [52]:
train_test_split_movies = user_vectors.randomSplit([0.6,0.4],123)
train_users = train_test_split_movies[0]
test_users = train_test_split_movies[1]
for k in [2,3,4,5,10,20]:
    k_model = KMeans.train(train_users,num_iterations,k,num_runs)
    cost = k_model.computeCost(test_users)
    print 'WCSS for k=%d : %f'%(k,cost)

WCSS for k=2 : 547.122121
WCSS for k=3 : 551.845096
WCSS for k=4 : 551.888517
WCSS for k=5 : 555.971549
WCSS for k=10 : 546.884437
WCSS for k=20 : 539.705653
